In [1]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2

In [2]:
page_x_inches: float = 8.5 # inches
page_y_inches: float = 11 # inches
border:float = 0.

perlin_grid_params = {
    'xstep':1,
    'ystep':1,
    'lod':10,
    'falloff':None,
    'noise_scale':0.0063,
    'noiseSeed':8
}
buffer_style = 3

In [5]:
@dataclass
class ScaleTransPrms(gp.DataClassBase):
    
    n_iters: int = 100
    d_buffer: float = -0.25
    d_translate_factor: float = 0.9
    d_translate: float = None
    angles: float = 0.
    d_translates: list = field(default=None, init=False)
    def __post_init__(self):
        self.d_buffers = np.array([self.d_buffer] * self.n_iters)
        
        if self.d_translates == None:
            if self.d_translate != None:
                self.d_translates =  np.array([self.d_translate] * self.n_iters)
            else:
                self.d_translates = self.d_buffers * self.d_translate_factor
    
    @property
    def prms(self):
        varnames = ['d_buffers', 'd_translates', 'angles']
        return {var: getattr(self, var) for var in varnames}

In [3]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds
brad = np.min([gp.get_width(drawbox), gp.get_height(drawbox)])

start_area = drawbox.centroid.buffer(brad*0.3)

In [16]:
stprm = ScaleTransPrms(
    d_buffer=-0.95,
    d_translate_factor=0.7,
    angles=45,
)

In [17]:
stprm.d_buffers += np.random.uniform(-0.3,0.3, stprm.d_buffers.shape)

In [18]:
ps = gp.scale_trans(start_area, **stprm.prms)

In [20]:
ls = gp.merge_LineStrings([p.boundary for p in ps])

In [21]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.geometry(ls)
sk.display(color_mode='layer')

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2020-12-03T09:22:55.621684 
 
 
 
 
 
 <polygon points="652.8,528.0 651.6212,504.0054 648.0962,480.2419 642.259,456.9383 634.1657,434.3191 623.8943,412.6021 611.5438,391.9964 597.233,372.7005 581.0997,354.9003 563.2995,338.767 544.0036,324.4562 523.3979,312.1057 501.6809,301.8343 479.0617,293.741 455.7581,287.9038 431.9946,284.3788 408.0,283.2 384.0054,284.3788 360.2419,287.9038 336.9383,293.741 314.3191,301.8343 292.6021,312.1057 271.9964,324.4562 252.7005,338.767 234.9003,354.9003 218.767,372.7005 204.4562,391.9964 192.1057,412.6021 181.8343,434.3191 173.741,456.9383 167.9038,480.2419 164.3788,504.0054 163.2,528.0 164.3788,551.9946 167.9038,575.7581 173.741,599.0617 181.8343,621.6809 192.1057,643.3979 204.4562,664.0036 218.767,683.2995 234.9003,701.0997 252.7005,717.233 271.9964,731.5438 292.6021,743.8943 314.3191,754.1657 336.9383,762.259 360.2419,768.0962 384.0054,771.6212 408.0,772.8 431.9946,771.6212 455.7581,768.0962 479.0617,762.259 501.6809,754.1657 523.3979,743.8943 544.0036,731.5438 563.2995,717.233 581.0997,701.0997 597.233,683.2995 611.5438,664.0036 623.8943,643.3979 634.1657,621.6809 642.259,599.0617 648.0962,575.7581 651.6212,551.9946 652.8,528.0"/>
 <polygon points="648.3301,525.8614 647.1665,502.1755 643.6869,478.7177 637.9247,455.7139 629.9356,433.3857 619.7963,411.9481 607.6047,391.6076 593.478,372.5599 577.5523,354.9887 559.9811,339.063 540.9335,324.9364 520.5929,312.7447 499.1553,302.6055 476.8271,294.6163 453.8233,288.8542 430.3655,285.3745 406.6797,284.2109 382.9938,285.3745 359.536,288.8542 336.5322,294.6163 314.204,302.6055 292.7664,312.7447 272.4259,324.9364 253.3783,339.063 235.807,354.9887 219.8814,372.5599 205.7547,391.6076 193.563,411.9481 183.4238,433.3857 175.4346,455.7139 169.6725,478.7177 166.1928,502.1755 165.0292,525.8614 166.1928,549.5472 169.6725,573.005 175.4346,596.0088 183.4238,618.337 193.563,639.7746 205.7547,660.1151 219.8814,679.1628 235.807,696.734 253.3783,712.6597 272.4259,726.7863 292.7664,738.978 314.204,749.1172 336.5322,757.1064 359.536,762.8685 382.9938,766.3482 406.6797,767.5118 430.3655,766.3482 453.8233,762.8685 476.8271,757.1064 499.1553,749.1172 520.5929,738.978 540.9335,726.7863 559.9811,712.6597 577.5523,696.734 593.478,679.1628 607.6047,660.1151 619.7963,639.7746 629.9356,618.337 637.9247,596.0088 643.6869,573.005 647.1665,549.5472"/>
 <polygon points="642.8321,523.7227 641.6886,500.4463 638.2691,477.3941 632.6066,454.788 624.7555,432.8458 614.7916,411.7788 602.8107,391.7899 588.9282,373.0716 573.2779,355.8041 556.0104,340.1538 537.2921,326.2713 517.3032,314.2904 496.2362,304.3265 474.294,296.4754 451.688,290.8129 428.6357,287.3934 405.3593,286.2499 382.0829,287.3934 359.0307,290.8129 336.4246,296.4754 314.4824,304.3265 293.4154,314.2904 273.4265,326.2713 254.7082,340.1538 237.4407,355.8041 221.7904,373.0716 207.9079,391.7899 195.9271,411.7788 185.9631,432.8458 178.1121,454.788 172.4495,477.3941 169.0301,500.4463 167.8866,523.7227 169.0301,546.9991 172.4495,570.0513 178.1121,592.6574 185.9631,614.5996 195.9271,635.6666 207.9079,655.6555 221.7904,674.3738 237.4407,691.6413 254.7082,707.2916 273.4265,721.1741 293.4154,733.155 314.4824,743.1189 336.4246,750.97 359.0307,756.6325 382.0829,760.052 405.3593,761.1955 428.6357,760.052 451.688,756.6325 474.294,750.97 496.2362,743.1189 517.3032,733.155 537.2921,721.1741 556.0104,707.2916 573.2779,691.6413 588.9282,674.3738 602.8107,655.6555 614.7916,635.6666 624.7555,614.5996 632.6066,592.6574 638.2691,570.0513 641.6886,546.9991"/>
 <polygon points="638.2216,521.5841 637.0939,498.6301 633.7218,475.8973 628.1377,453.6044 620.3955,431.9663 610.5696,411.1912 598.7547,391.4792 585.0646,373.0202 569.6311,355.992 552.6028,340.5585 534.1439,326.8684 514.4319,315.0535 493.6568,305.2276 472.0186,297.4853 449.7257,291.9012 426.9929,288.5291 404.039,287.4015 381.0851,288.5291 358.3522,291.9012 336.0594,297.4853 314.4212,305.2276 293.6461,315.0535 273.9341,326.8684 255.4751

In [22]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0089_literally_just_a_circle.svg'

sk.save(savepath)

vpype_commands = 'linesimplify --tolerance 0.2mm linemerge --tolerance 0.2mm reloop linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

512

In [2]:
page_x_inches: float = 6 # inches
page_y_inches: float = 6 # inches
border:float = 0.

perlin_grid_params = {
    'xstep':1,
    'ystep':1,
    'lod':10,
    'falloff':None,
    'noise_scale':0.0063,
    'noiseSeed':8
}
buffer_style = 3

In [3]:
@dataclass
class ScaleTransPrms(gp.DataClassBase):
    
    n_iters: int = 100
    d_buffer: float = -0.25
    d_translate_factor: float = 0.9
    d_translate: float = None
    angles: float = 0.
    d_translates: list = field(default=None, init=False)
    def __post_init__(self):
        self.d_buffers = np.array([self.d_buffer] * self.n_iters)
        
        if self.d_translates == None:
            if self.d_translate != None:
                self.d_translates =  np.array([self.d_translate] * self.n_iters)
            else:
                self.d_translates = self.d_buffers * self.d_translate_factor
    
    @property
    def prms(self):
        varnames = ['d_buffers', 'd_translates', 'angles']
        return {var: getattr(self, var) for var in varnames}

In [10]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds
brad = np.min([gp.get_width(drawbox), gp.get_height(drawbox)])

start_area = drawbox.centroid.buffer(brad*0.45)

In [24]:
stprm = ScaleTransPrms(
    d_buffer=-0.25,
    d_translate_factor=0.7,
    angles=45,
    n_iters=200
)

In [25]:
stprm.d_buffers += np.random.uniform(-0.07,0.1, stprm.d_buffers.shape)

In [26]:
ps = gp.scale_trans(start_area, **stprm.prms)

In [27]:
ls = gp.merge_LineStrings([p.boundary for p in ps])

In [28]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.geometry(ls)
sk.display(color_mode='layer')

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2020-12-03T22:25:23.474299 
 
 
 
 
 
 <polygon points="547.2,288.0 545.9519,262.594 542.2195,237.4326 536.0389,212.7582 527.4696,188.8085 516.594,165.814 503.5169,143.9962 488.3643,123.5653 471.2821,104.7179 452.4347,87.6357 432.0038,72.4831 410.186,59.406 387.1915,48.5304 363.2418,39.9611 338.5674,33.7805 313.406,30.0481 288.0,28.8 262.594,30.0481 237.4326,33.7805 212.7582,39.9611 188.8085,48.5304 165.814,59.406 143.9962,72.4831 123.5653,87.6357 104.7179,104.7179 87.6357,123.5653 72.4831,143.9962 59.406,165.814 48.5304,188.8085 39.9611,212.7582 33.7805,237.4326 30.0481,262.594 28.8,288.0 30.0481,313.406 33.7805,338.5674 39.9611,363.2418 48.5304,387.1915 59.406,410.186 72.4831,432.0038 87.6357,452.4347 104.7179,471.2821 123.5653,488.3643 143.9962,503.5169 165.814,516.594 188.8085,527.4696 212.7582,536.0389 237.4326,542.2195 262.594,545.9519 288.0,547.2 313.406,545.9519 338.5674,542.2195 363.2418,536.0389 387.1915,527.4696 410.186,516.594 432.0038,503.5169 452.4347,488.3643 471.2821,471.2821 488.3643,452.4347 503.5169,432.0038 516.594,410.186 527.4696,387.1915 536.0389,363.2418 542.2195,338.5674 545.9519,313.406 547.2,288.0"/>
 <polygon points="545.7693,287.4372 544.5264,262.1373 540.8097,237.0811 534.6549,212.5099 526.1213,188.6602 515.2912,165.7618 502.2688,144.0352 487.1795,123.6897 470.1687,104.9211 451.4001,87.9102 431.0545,72.8209 409.3279,59.7985 386.4296,48.9684 362.5799,40.4348 338.0086,34.2801 312.9524,30.5633 287.6525,29.3204 262.3527,30.5633 237.2965,34.2801 212.7252,40.4348 188.8755,48.9684 165.9771,59.7985 144.2505,72.8209 123.905,87.9102 105.1364,104.9211 88.1256,123.6897 73.0363,144.0352 60.0139,165.7618 49.1837,188.6602 40.6502,212.5099 34.4954,237.0811 30.7787,262.1373 29.5358,287.4372 30.7787,312.7371 34.4954,337.7933 40.6502,362.3645 49.1837,386.2142 60.0139,409.1126 73.0363,430.8392 88.1256,451.1847 105.1364,469.9533 123.905,486.9642 144.2505,502.0534 165.9771,515.0759 188.8755,525.906 212.7252,534.4395 237.2965,540.5943 262.3527,544.3111 287.6525,545.554 312.9524,544.3111 338.0086,540.5943 362.5799,534.4395 386.4296,525.906 409.3279,515.0759 431.0545,502.0534 451.4001,486.9642 470.1687,469.9533 487.1795,451.1847 502.2688,430.8392 515.2912,409.1126 526.1213,386.2142 534.6549,362.3645 540.8097,337.7933 544.5264,312.7371"/>
 <polygon points="544.4961,286.8744 543.2576,261.6653 539.5542,236.6989 533.4215,212.2158 524.9186,188.4517 514.1273,165.6354 501.1516,143.9867 486.1164,123.7142 469.1666,105.0129 450.4653,88.0631 430.1927,73.0279 408.5441,60.0522 385.7278,49.2609 361.9637,40.758 337.4806,34.6253 312.5142,30.9219 287.3051,29.6834 262.096,30.9219 237.1296,34.6253 212.6465,40.758 188.8824,49.2609 166.0661,60.0522 144.4174,73.0279 124.1449,88.0631 105.4436,105.0129 88.4938,123.7142 73.4586,143.9867 60.4829,165.6354 49.6916,188.4517 41.1887,212.2158 35.056,236.6989 31.3526,261.6653 30.1141,286.8744 31.3526,312.0835 35.056,337.0499 41.1887,361.533 49.6916,385.2971 60.4829,408.1134 73.4586,429.762 88.4938,450.0346 105.4436,468.7359 124.1449,485.6857 144.4174,500.7209 166.0661,513.6966 188.8824,524.4879 212.6465,532.9908 237.1296,539.1235 262.096,542.8269 287.3051,544.0654 312.5142,542.8269 337.4806,539.1235 361.9637,532.9908 385.7278,524.4879 408.5441,513.6966 430.1927,500.7209 450.4653,485.6857 469.1666,468.7359 486.1164,450.0346 501.1516,429.762 514.1273,408.1134 524.9186,385.2971 533.4215,361.533 539.5542,337.0499 543.2576,312.0835"/>
 <polygon points="543.2568,286.3116 542.0226,261.1899 538.3321,236.3101 532.2206,211.9119 523.7472,188.2302 512.9933,165.493 500.0626,143.9194 485.0796,123.7171 468.1885,105.0807 449.5521,88.1897 429.3498,73.2066 407.7762,60.2759 385.0391,49.5221 361.3573,41.0486 336.9591,34.9372 312.0793,31.2466 286.9576,30.0124 261.8359,31.2466 236.9561,34.9372 212.5579,41.0486 188.8762,49.5221 166.139,60.2759 144.5654,73.2066 124.3632,88.1897 105.7268,105.0807 88.8357,123.7171 73.8527,143.9194 60.922,165.493 50.1681,188.2302 

In [29]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0089_literally_just_a_circle.svg'

sk.save(savepath)

vpype_commands = 'linesimplify --tolerance 0.05mm reloop linemerge --tolerance 0.05mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

512